In [ ]:
#####  RUN NOTES
# Justice Conformity study3, HDDM analysis
# Written by Jae-Young Son
# Last modified 08-13-20 for HDDM workshop (Carney Computational Modeling Workshop)


#####  SET UP ENVIRONMENT

# Import requisite packages
import hddm
import pandas as pd
import pickle
from patsy import dmatrix
from kabuki.analyze import gelman_rubin
from kabuki.utils import concat_models
import pathlib

# Name this model
modelName = 'categorical_unfair_recovery20'  # Change this!

# Get around a problem with saving regression outputs in Python 3
def savePatch(self, fname):
    import pickle
    with open(fname, 'wb') as f:
        pickle.dump(self, f)
hddm.HDDM.savePatch = savePatch

In [ ]:
#####  SIMULATE DATA BASED ON PARAMETERS ESTIMATED FROM OBSERVED DATA

# Number of subjects matches that of the actual experiment
# For the number of trials per condition, you should run this twice:
#      Once with a small number of trials (or, the number of trials in your experiment), which lets you know whether
#           HDDM can reliably estimate the parameters given the amount of data you originally fed it
#      Once with a larger number of trials, which lets you know whether HDDM can reliably estimate the parameters as
#           the amount of data used to inform the computation approaches infinity
num_subs = 40
trials_per_cond = 20

# Drift rate estimates (empirical)
v_A = -0.2006909
v_0 = v_A + -0.7693828
v_1 = v_A + -0.6429486
v_2 = v_A + -0.1640435
v_3 = v_A + 0.4457329
v_4 = v_A + 0.6793762

# Threshold estimates
a_A = 2.1356529
a_0 = a_A + -0.1365446
a_1 = a_A + -0.1131092
a_2 = a_A + 0.0615770
a_3 = a_A + -0.0230197
a_4 = a_A + -0.1731338

# Bias and nondecision time estimates
z = 0.4410610
t = 0.3206254

# Define each condition and its corresponding parameters
params = {
    'rev_A': {'a':a_A, 't':t, 'v':v_A, 'z':z},
    'rev_0': {'a':a_0, 't':t, 'v':v_0, 'z':z},
    'rev_1': {'a':a_1, 't':t, 'v':v_1, 'z':z},
    'rev_2': {'a':a_2, 't':t, 'v':v_2, 'z':z},
    'rev_3': {'a':a_3, 't':t, 'v':v_3, 'z':z},
    'rev_4': {'a':a_4, 't':t, 'v':v_4, 'z':z}
}

# Now simulate that data
data, params = hddm.generate.gen_rand_data(
    params,
    size=trials_per_cond,
    subjs=num_subs,
    subj_noise=.1
)

In [ ]:
#####  ESTIMATE SIMULATED PARAMETERS USING THE SAME REGRESSION MODEL
#####  USED TO ESTIMATE OBSERVED PARAMETERS

# Notes: Literally just copy/paste the regression model you used to estimate parameters. The point of doing this
#        analysis is to make sure that the combination of parameters you estimated from the empirical data are
#        parameters that HDDM is capable of reliably estimating. So given that we know exactly what the expected
#        parameter values are (since we simulated them), can HDDM then take that simulated data and give us an
#        estimate that's in the right ballpark?


# Create empty array that will eventually store our models
models = []

# Loop over 5 times to get 5 chains
for i in range(5):
    # Define regression model
    m = hddm.HDDMRegressor(data,
                           # Change this!
                           {"v ~ C(rev_cat, Treatment('Alone'))",
                            "a ~ C(rev_cat, Treatment('Alone'))"},
                           group_only_regressors=True,
                           p_outlier=.05,
                           include={'z'})
    m.find_starting_values()
    
    # Start sampling from the defined regression model
    #      For the sake of demonstration, we only collect 10 traces (not nearly enough)
    #      It's also conventional to discard/"burn" the first X traces, as they're typically not good estimates
    #      Instead of storing traces in RAM, we're going to save them to our hard drive
    m.sample(10, burn=0, dbname='./Models/'+modelName+'_%s.db'%i, db='pickle')
    m.savePatch('./Models/'+modelName+'_%s'%i)
    
    # Once you're finished running a chain, add that chain to your array of models
    models.append(m)

In [ ]:
#####  GET STATS

# Calculate Gelman-Rubin r-hat statistic
m_rhat = gelman_rubin(models)
pd.DataFrame.from_dict(m_rhat, orient='index').to_csv('./Results/'+modelName+'_RHat.csv')

# Save traces of concatenated model (only valid to look at if converged!)
m_comb = concat_models(models)
m_comb_export = m_comb.get_traces()
m_comb_export.to_csv('./Results/'+modelName+'_traces.csv')